Analysis of a used cars advertisements on a database from may 2018 to April 2019.


On this practice I will be creating a web service displaying different plots analyzing the information from a used cars advertisements database.
The end goal is to host the web service on Render and to be able to display interative plots without issues. To that end we will be using GitHub repository and the elements of the webpage will be created using Streamlit/Pyplot express.
All of the information to display in the final web service will be first tested locally using VS code and later commiting the changes to GitHub via the command line.

1.0 Describing the process



The analysis will start by loading all the necessary libraries and the database into Pandas. Then we will perform some exploratory analysis: dealing with duplicates and missing values. Following with enriching the data. Then we will be ready to create our plots to be displayed and finally we will create the rest of the elements that will be needed for the web service.

1.1 Initialization

In [179]:
#Importing all the necessary libraries 
import pandas as pd 
import plotly.express as px
import streamlit as st
import numpy as np
import altair as alt

1.2 Loading data

In [180]:
#Reading the df into pandas and getting a sample of the information.
df=pd.read_csv("C:/Users/Owner/used_cars_new/vehicles_us.csv")
df.sample(10)

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
2102,8995,2013.0,nissan rogue,good,4.0,gas,76903.0,automatic,SUV,grey,NaN,2019-03-20,60
11655,12995,2016.0,ford fusion se,excellent,4.0,gas,80388.0,automatic,sedan,silver,NaN,2019-02-09,32
26784,14900,2014.0,ford edge,excellent,6.0,gas,NaN,automatic,SUV,white,1.0,2019-03-14,16
25333,39988,2018.0,toyota tacoma,like new,6.0,gas,15000.0,manual,truck,black,1.0,2018-05-16,58
31992,12995,2012.0,acura tl,like new,6.0,gas,94121.0,automatic,sedan,NaN,1.0,2018-07-17,22
2227,11500,2005.0,ford f-250,good,8.0,diesel,151350.0,automatic,truck,brown,1.0,2019-04-04,16
6694,23000,2011.0,ford f350 super duty,excellent,8.0,diesel,NaN,automatic,pickup,silver,1.0,2019-01-13,17
35715,22999,2011.0,ford f350 super duty,excellent,8.0,diesel,130648.0,automatic,pickup,white,1.0,2019-03-06,25
2040,16495,2010.0,ford f-150,excellent,8.0,gas,79776.0,automatic,truck,black,1.0,2018-11-30,10
6207,2495,2001.0,ford taurus,good,6.0,gas,165000.0,automatic,sedan,silver,NaN,2018-09-13,9


After a first glimpse at the information, it is apparent that there is an important amount of empty values to deal with. I'll run the info method to get a better understanding of the situation:

In [181]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         51525 non-null  int64  
 1   model_year    47906 non-null  float64
 2   model         51525 non-null  object 
 3   condition     51525 non-null  object 
 4   cylinders     46265 non-null  float64
 5   fuel          51525 non-null  object 
 6   odometer      43633 non-null  float64
 7   transmission  51525 non-null  object 
 8   type          51525 non-null  object 
 9   paint_color   42258 non-null  object 
 10  is_4wd        25572 non-null  float64
 11  date_posted   51525 non-null  object 
 12  days_listed   51525 non-null  int64  
dtypes: float64(4), int64(2), object(7)
memory usage: 5.1+ MB


We encounter missing values in the columns: model_year, cylinders, odometer, paint_color, and is_4wd. 
But before jumping into dealing with the empty values, it would be relevant to check for duplicated rows. 

1.3 Preparing the data


In this section I will look for duplicates, make sure that the values in each column are the right type for the analysis that will be done and I will deal with missing values.

In [182]:
#Looking for duplicates
df.duplicated().sum()

0

In [183]:
#Making sure there are no duplicates by calling unique values on some columns:
df['model'].unique()

array(['bmw x5', 'ford f-150', 'hyundai sonata', 'chrysler 200',
       'chrysler 300', 'toyota camry', 'honda pilot', 'kia sorento',
       'chevrolet silverado 1500', 'honda accord', 'ram 1500',
       'gmc yukon', 'jeep cherokee', 'chevrolet traverse',
       'hyundai elantra', 'chevrolet tahoe', 'toyota rav4',
       'chevrolet silverado', 'jeep wrangler', 'chevrolet malibu',
       'ford fusion se', 'chevrolet impala', 'chevrolet corvette',
       'jeep liberty', 'toyota camry le', 'nissan altima',
       'subaru outback', 'toyota highlander', 'dodge charger',
       'toyota tacoma', 'chevrolet equinox', 'nissan rogue',
       'mercedes-benz benze sprinter 2500', 'honda cr-v',
       'jeep grand cherokee', 'toyota 4runner', 'ford focus',
       'honda civic', 'kia soul', 'chevrolet colorado',
       'ford f150 supercrew cab xlt', 'chevrolet camaro lt coupe 2d',
       'chevrolet cruze', 'ford mustang', 'chevrolet silverado 3500hd',
       'nissan frontier crew cab sv', 'subaru imp

In [184]:
#Making sure there are no duplicates by calling unique values on some columns:
df['type'].unique()

array(['SUV', 'pickup', 'sedan', 'truck', 'coupe', 'van', 'convertible',
       'hatchback', 'wagon', 'mini-van', 'other', 'offroad', 'bus'],
      dtype=object)

So far it doesn't seem like there are duplicated rows. Also, the results of the unique method, show that there don't seem to be variations of the same word with different upper or lowercase letters. 
Next, I'll check that the type of the data on each column makes sense. In general everything looks good. The only improvement that could be done, would be to convert the "date_posted" column from a general object type to an actual datetime object. 

In [ ]:
#Converting the "date_posted" column to datetime object.
df['date_posted']=pd.to_datetime(df['date_posted'], format='%Y-%m-%d')
#Finding out the range of dates considered on this database
print(df['date_posted'].min())
print(df['date_posted'].max())
#Confirming that the column was transformed correctly:
df.info()

2018-05-01 00:00:00
2019-04-19 00:00:00
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   price           51525 non-null  int64         
 1   model_year      51525 non-null  int32         
 2   model           51525 non-null  object        
 3   condition       51525 non-null  object        
 4   cylinders       51525 non-null  float64       
 5   fuel            51525 non-null  object        
 6   odometer        51524 non-null  float64       
 7   transmission    51525 non-null  object        
 8   type            51525 non-null  object        
 9   paint_color     51525 non-null  object        
 10  is_4wd          51525 non-null  float64       
 11  date_posted     51525 non-null  datetime64[ns]
 12  days_listed     51525 non-null  int64         
 13  make            51525 non-null  object        
 14  month_posted  

It seems like it also would be relevant to add a column with the make, this will be useful when dealing with missing values. 

In [186]:
#Creating a new column containing the make only. The reasoning here is that the model seems to be the first word of each value, so the first word is being extracted to this new column.
df['make'] = df['model'].str.split().str[0]
#Getting a sample with this new column. I ran it several times to get an idea of whether it was done correctly. It seems like it is.
df.sample(10)

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed,make
31296,5400,2010.0,volkswagen passat,like new,NaN,gas,117700.0,automatic,sedan,blue,NaN,2018-08-15,66,volkswagen
45658,10884,2014.0,ford escape,like new,6.0,gas,91999.0,automatic,sedan,red,NaN,2019-03-31,57,ford
37243,8495,2010.0,honda cr-v,excellent,4.0,gas,124722.0,automatic,SUV,brown,NaN,2019-01-18,36,honda
7315,35800,2015.0,ram 2500,like new,6.0,diesel,84000.0,automatic,truck,blue,1.0,2018-07-15,47,ram
46673,11988,2007.0,ford ranger,like new,6.0,gas,NaN,automatic,truck,white,1.0,2019-04-17,60,ford
6522,7495,2010.0,honda pilot,good,6.0,gas,121020.0,automatic,other,brown,1.0,2018-09-28,19,honda
1881,3500,2009.0,dodge grand caravan,good,6.0,gas,195000.0,automatic,mini-van,silver,NaN,2018-07-22,31,dodge
8020,7910,2010.0,chrysler town & country,excellent,6.0,gas,103065.0,automatic,mini-van,red,NaN,2018-10-19,29,chrysler
18866,2000,2006.0,hyundai elantra,good,4.0,gas,NaN,automatic,sedan,red,NaN,2018-06-20,24,hyundai
32551,5995,1993.0,ford f-250,good,8.0,diesel,302097.0,automatic,truck,grey,1.0,2018-08-08,17,ford


It is finally time to deal with the missing values. I will group the information in regards to another relevant column and then the missing values will be filled with the mean/median/mode  of the correspondent group.

In [187]:
#The first column with missing values is the model_year. The df will be grouped by the make and the mean of the year of the vehicles of each make will be assigned as the new model_year with the transform method.
# The end result will be transformed from a float object to an integer since there are not decimals involved/needed in the year information.
#Dealing with measing values on 'model_year"
df['model_year'] = df.groupby('make')['model_year'].transform(lambda x: x.fillna(x.mode().iloc[0]))
df['model_year'] = df['model_year'].astype(int)
#Getting a sample of the data after the adjustments.
df.info()
df.sample(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   price         51525 non-null  int64         
 1   model_year    51525 non-null  int32         
 2   model         51525 non-null  object        
 3   condition     51525 non-null  object        
 4   cylinders     46265 non-null  float64       
 5   fuel          51525 non-null  object        
 6   odometer      43633 non-null  float64       
 7   transmission  51525 non-null  object        
 8   type          51525 non-null  object        
 9   paint_color   42258 non-null  object        
 10  is_4wd        25572 non-null  float64       
 11  date_posted   51525 non-null  datetime64[ns]
 12  days_listed   51525 non-null  int64         
 13  make          51525 non-null  object        
dtypes: datetime64[ns](1), float64(3), int32(1), int64(2), object(7)
memory usage: 5.3+ MB


,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed,make
16071,700,2016,toyota camry,like new,4.0,gas,12989.0,automatic,sedan,black,NaN,2018-12-25,21,toyota
37475,10500,2012,dodge charger,good,6.0,gas,114000.0,automatic,sedan,black,1.0,2018-09-05,3,dodge
1908,5900,2004,toyota tacoma,excellent,4.0,gas,92000.0,manual,pickup,silver,NaN,2018-09-22,9,toyota
29843,4495,2003,chevrolet trailblazer,excellent,6.0,gas,145515.0,automatic,SUV,green,1.0,2019-04-14,43,chevrolet
33274,22900,2013,ford f-350 sd,good,8.0,diesel,172926.0,automatic,truck,white,1.0,2019-04-17,36,ford
49228,4995,2009,ford escape,good,4.0,gas,100612.0,automatic,SUV,NaN,NaN,2018-05-21,27,ford
892,5995,2010,toyota corolla,excellent,4.0,gas,144000.0,automatic,sedan,red,NaN,2018-07-12,66,toyota
33356,16500,2016,chevrolet colorado,excellent,6.0,gas,143276.0,automatic,pickup,silver,1.0,2018-07-31,87,chevrolet
5237,2999,2001,volkswagen passat,like new,4.0,gas,114982.0,automatic,sedan,silver,NaN,2018-08-24,79,volkswagen
13628,8950,2008,dodge charger,like new,8.0,gas,129141.0,automatic,sedan,red,NaN,2019-02-22,43,dodge


In [188]:
#The next column with missing values is the cylinders one. For this, the relevant information to compare it to, is in the 'type' column. Same process will be followed, except that in this occassion instead of the mean, the median will be assigned to prevent any outliers from affecting the result.

#Dealing with missing values on 'cylinders' column
df['cylinders'] = df.groupby('type')['cylinders'].transform(lambda x: x.fillna(x.median()))
df.info()
df.sample(10)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   price         51525 non-null  int64         
 1   model_year    51525 non-null  int32         
 2   model         51525 non-null  object        
 3   condition     51525 non-null  object        
 4   cylinders     51525 non-null  float64       
 5   fuel          51525 non-null  object        
 6   odometer      43633 non-null  float64       
 7   transmission  51525 non-null  object        
 8   type          51525 non-null  object        
 9   paint_color   42258 non-null  object        
 10  is_4wd        25572 non-null  float64       
 11  date_posted   51525 non-null  datetime64[ns]
 12  days_listed   51525 non-null  int64         
 13  make          51525 non-null  object        
dtypes: datetime64[ns](1), float64(3), int32(1), int64(2), object(7)
memory usage: 5.3+ MB


,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed,make
44514,31900,2015,chevrolet silverado 2500hd,good,8.0,diesel,154613.0,automatic,truck,black,1.0,2019-01-28,31,chevrolet
3936,7000,2000,ford mustang,like new,6.0,gas,199000.0,manual,coupe,blue,NaN,2018-05-24,86,ford
5135,19985,2006,ram 3500,excellent,6.0,diesel,NaN,automatic,pickup,NaN,1.0,2018-09-11,101,ram
385,27995,2017,chevrolet silverado 1500,excellent,8.0,gas,72965.0,automatic,truck,black,1.0,2018-07-23,42,chevrolet
41242,33000,2015,toyota 4runner,excellent,6.0,gas,70100.0,automatic,SUV,white,1.0,2018-08-21,64,toyota
723,23999,2013,jeep wrangler unlimited,excellent,6.0,gas,136357.0,automatic,convertible,NaN,1.0,2019-02-11,24,jeep
8452,5995,2010,toyota corolla,excellent,4.0,gas,144000.0,automatic,sedan,red,NaN,2018-07-24,48,toyota
44201,7999,2013,chevrolet silverado,good,6.0,gas,181000.0,automatic,pickup,white,NaN,2018-05-04,12,chevrolet
6112,5900,2016,chevrolet impala,good,4.0,gas,103.0,automatic,sedan,black,NaN,2018-11-29,9,chevrolet
50129,20000,2018,subaru forester,excellent,4.0,gas,31000.0,automatic,SUV,white,1.0,2018-06-18,23,subaru


In [189]:
#The same process will be done with the 'odometer' column. In this case the relevant information will be the model year, assuming that years with the same age tend to have a similar use.
#Dealing with missing values in 'odometer' column
df['odometer'] = df.groupby('model_year')['odometer'].transform(lambda x: x.fillna(x.mean()))
df['odometer'] = df['odometer'].round(0)
df.info()
#For some reason there is still one missing value on the odometer column. Here it is.
print(df[df['odometer'].isnull()])
#I'm thinking it didn't get a mean because there are not other cars from year 1929. For now I'll leave it as it is since one value won't affect the calculations and I think it would be interesting to see in the future how many cars as old as this one were sold. I'm thinking not many, and losing one of them could be relevant.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   price         51525 non-null  int64         
 1   model_year    51525 non-null  int32         
 2   model         51525 non-null  object        
 3   condition     51525 non-null  object        
 4   cylinders     51525 non-null  float64       
 5   fuel          51525 non-null  object        
 6   odometer      51524 non-null  float64       
 7   transmission  51525 non-null  object        
 8   type          51525 non-null  object        
 9   paint_color   42258 non-null  object        
 10  is_4wd        25572 non-null  float64       
 11  date_posted   51525 non-null  datetime64[ns]
 12  days_listed   51525 non-null  int64         
 13  make          51525 non-null  object        
dtypes: datetime64[ns](1), float64(3), int32(1), int64(2), object(7)
memory usage: 5.3+ MB


In [ ]:
#Dealing with missing values in 'paint_color' column. It is my impression that makes tend to have an available 'palette' that they use among their different vehicles.
#I'll be grouping the information according to the make and will fill in the missing values using the mode value.
group_mode = df.groupby('make')['paint_color'].apply(lambda x: x.mode()[0])
df['paint_color'] = df.groupby('make')['paint_color'].transform(lambda x: x.fillna(group_mode[x.name]))
df.info()
df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   price         51525 non-null  int64         
 1   model_year    51525 non-null  int32         
 2   model         51525 non-null  object        
 3   condition     51525 non-null  object        
 4   cylinders     51525 non-null  float64       
 5   fuel          51525 non-null  object        
 6   odometer      51524 non-null  float64       
 7   transmission  51525 non-null  object        
 8   type          51525 non-null  object        
 9   paint_color   51525 non-null  object        
 10  is_4wd        25572 non-null  float64       
 11  date_posted   51525 non-null  datetime64[ns]
 12  days_listed   51525 non-null  int64         
 13  make          51525 non-null  object        
dtypes: datetime64[ns](1), float64(3), int32(1), int64(2), object(7)
memory usage: 5.3+ MB


,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed,make
0,9400,2011,bmw x5,good,6.0,gas,145000.0,automatic,SUV,black,1.0,2018-06-23,19,bmw
1,25500,2013,ford f-150,good,6.0,gas,88705.0,automatic,pickup,white,1.0,2018-10-19,50,ford
2,5500,2013,hyundai sonata,like new,4.0,gas,110000.0,automatic,sedan,red,NaN,2019-02-07,79,hyundai
3,1500,2003,ford f-150,fair,8.0,gas,162072.0,automatic,pickup,white,NaN,2019-03-22,9,ford
4,14900,2017,chrysler 200,excellent,4.0,gas,80903.0,automatic,sedan,black,NaN,2019-04-02,28,chrysler
5,14990,2014,chrysler 300,excellent,6.0,gas,57954.0,automatic,sedan,black,1.0,2018-06-20,15,chrysler
6,12990,2015,toyota camry,excellent,4.0,gas,79212.0,automatic,sedan,white,NaN,2018-12-27,73,toyota
7,15990,2013,honda pilot,excellent,6.0,gas,109473.0,automatic,SUV,black,1.0,2019-01-07,68,honda
8,11500,2012,kia sorento,excellent,4.0,gas,104174.0,automatic,SUV,white,1.0,2018-07-16,19,kia
9,9200,2008,honda pilot,excellent,6.0,gas,147191.0,automatic,SUV,blue,1.0,2019-02-15,17,honda


Finally, we are left to deal with the missing values on the is_4wd column.
At a first glimse I have the impresion that in this case the way they represented if it is or isn't a 4wd is either with 1 for 'True' and NaN for 'False'. To test this hypothesis I will print the unique values in the column.

In [192]:
#Printing unique values in is_4wd column:
df['is_4wd'].unique()

array([ 1., nan])

As expected, there are only two values in the 'is_4wd' column. I think it will be convenient to simply represent the vehicles that are not 4wd with a 0. 

In [193]:
#Replacing missing values in 'is_4wd' with '0'
df['is_4wd'] = df['is_4wd'].fillna(0)
#Verifying that we dealt with all missing values. Except for the one we left empty on purpose.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   price         51525 non-null  int64         
 1   model_year    51525 non-null  int32         
 2   model         51525 non-null  object        
 3   condition     51525 non-null  object        
 4   cylinders     51525 non-null  float64       
 5   fuel          51525 non-null  object        
 6   odometer      51524 non-null  float64       
 7   transmission  51525 non-null  object        
 8   type          51525 non-null  object        
 9   paint_color   51525 non-null  object        
 10  is_4wd        51525 non-null  float64       
 11  date_posted   51525 non-null  datetime64[ns]
 12  days_listed   51525 non-null  int64         
 13  make          51525 non-null  object        
dtypes: datetime64[ns](1), float64(3), int32(1), int64(2), object(7)
memory usage: 5.3+ MB


1.4 Enriching data
In this section I will create some extra columns that might be relevant for the analysis.

In [206]:
#First I will create a new column containing the day, month and year the product was posted:
df['day_posted']=df['date_posted'].dt.day
df['month_posted']=df['date_posted'].dt.month
df['year_posted']=df['date_posted'].dt.year
print(df['year_posted'].unique())
df.sample(10)

[2018 2019]


,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,...,date_posted,days_listed,make,month_posted,year_posted,date_purchase,month_purchase,day_purchase,classification,day_posted
17570,15900,2008,ford f-250 sd,good,8.0,diesel,107374.0,automatic,truck,grey,...,2018-09-20,26,ford,9,2018,2018-10-16,10,16,modern,20
22683,10950,2012,ford explorer,excellent,6.0,gas,121453.0,automatic,SUV,silver,...,2018-06-20,66,ford,6,2018,2018-08-25,8,25,late_model,20
48437,5900,2013,ford f-150,excellent,6.0,gas,178740.0,automatic,truck,white,...,2018-06-28,50,ford,6,2018,2018-08-17,8,17,late_model,28
10422,4995,2009,dodge charger,good,6.0,gas,132707.0,automatic,sedan,black,...,2018-06-18,43,dodge,6,2018,2018-07-31,7,31,late_model,18
21164,11995,2011,ram 1500,good,8.0,gas,116861.0,automatic,truck,white,...,2019-01-12,21,ram,1,2019,2019-02-02,2,2,late_model,12
27265,9995,2015,nissan altima,good,4.0,gas,81978.0,automatic,sedan,silver,...,2018-10-28,24,nissan,10,2018,2018-11-21,11,21,late_model,28
32960,8000,2002,chevrolet silverado 1500,excellent,8.0,gas,140000.0,automatic,truck,green,...,2019-03-08,12,chevrolet,3,2019,2019-03-20,3,20,modern,8
27232,4200,2000,ford mustang,good,8.0,gas,49520.0,automatic,convertible,white,...,2019-02-24,20,ford,2,2019,2019-03-16,3,16,modern,24
45558,14995,2011,ford ranger,excellent,6.0,gas,125058.0,automatic,truck,grey,...,2018-05-23,11,ford,5,2018,2018-06-03,6,3,late_model,23
23272,11990,2012,buick enclave,good,6.0,gas,84148.0,automatic,SUV,white,...,2018-11-01,62,buick,11,2018,2019-01-02,1,2,late_model,1


In [195]:
#I thought it would also be interesting to see the month when the car was bought in the end. Maybe we can find some relevant information regarding the buying behavior.
#I am assuming that 'days_posted' means that the car was purchased on that date. But it could also mean that the person simply took it down. This would need to be confirmed by the provider of the information.
df['date_purchase'] = df['date_posted'] + pd.to_timedelta(df['days_listed'], unit='D')
df.sample(10)

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed,make,month_posted,year_posted,date_purchase
39935,9500,2011,chevrolet silverado,good,8.0,gas,125058.0,automatic,truck,silver,0.0,2018-08-29,6,chevrolet,8,2018,2018-09-04
6107,14995,2005,jeep wrangler,excellent,6.0,gas,97039.0,manual,SUV,black,1.0,2018-07-15,13,jeep,7,2018,2018-07-28
38932,9995,2011,toyota rav4,good,6.0,gas,127677.0,automatic,SUV,black,1.0,2019-01-29,41,toyota,1,2019,2019-03-11
23760,13999,2018,hyundai elantra,like new,4.0,gas,17793.0,automatic,sedan,silver,0.0,2019-03-01,45,hyundai,3,2019,2019-04-15
35329,7800,2007,subaru impreza,good,4.0,gas,142043.0,manual,sedan,black,1.0,2018-10-03,84,subaru,10,2018,2018-12-26
16658,9995,2013,volkswagen passat,excellent,5.0,gas,85010.0,automatic,sedan,black,0.0,2019-03-12,51,volkswagen,3,2019,2019-05-02
32163,8999,2004,toyota sienna,excellent,6.0,gas,99946.0,automatic,mini-van,silver,1.0,2018-12-14,5,toyota,12,2018,2018-12-19
4117,16000,2014,chevrolet camaro lt coupe 2d,good,6.0,gas,73289.0,other,coupe,black,0.0,2019-02-07,24,chevrolet,2,2019,2019-03-03
19268,49950,2018,ram 2500,like new,8.0,diesel,4800.0,automatic,truck,grey,1.0,2018-08-15,129,ram,8,2018,2018-12-22
12197,17700,2015,nissan frontier crew cab sv,good,6.0,gas,67071.0,other,pickup,black,0.0,2019-01-23,78,nissan,1,2019,2019-04-11


In [196]:
#With the 'date_purchase' column, now we can also know the month when a car was bought and the day of the month.
df['month_purchase']=df['date_purchase'].dt.month
df['day_purchase']=df['date_purchase'].dt.day
df.head(10)

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed,make,month_posted,year_posted,date_purchase,month_purchase,day_purchase
0,9400,2011,bmw x5,good,6.0,gas,145000.0,automatic,SUV,black,1.0,2018-06-23,19,bmw,6,2018,2018-07-12,7,12
1,25500,2013,ford f-150,good,6.0,gas,88705.0,automatic,pickup,white,1.0,2018-10-19,50,ford,10,2018,2018-12-08,12,8
2,5500,2013,hyundai sonata,like new,4.0,gas,110000.0,automatic,sedan,red,0.0,2019-02-07,79,hyundai,2,2019,2019-04-27,4,27
3,1500,2003,ford f-150,fair,8.0,gas,162072.0,automatic,pickup,white,0.0,2019-03-22,9,ford,3,2019,2019-03-31,3,31
4,14900,2017,chrysler 200,excellent,4.0,gas,80903.0,automatic,sedan,black,0.0,2019-04-02,28,chrysler,4,2019,2019-04-30,4,30
5,14990,2014,chrysler 300,excellent,6.0,gas,57954.0,automatic,sedan,black,1.0,2018-06-20,15,chrysler,6,2018,2018-07-05,7,5
6,12990,2015,toyota camry,excellent,4.0,gas,79212.0,automatic,sedan,white,0.0,2018-12-27,73,toyota,12,2018,2019-03-10,3,10
7,15990,2013,honda pilot,excellent,6.0,gas,109473.0,automatic,SUV,black,1.0,2019-01-07,68,honda,1,2019,2019-03-16,3,16
8,11500,2012,kia sorento,excellent,4.0,gas,104174.0,automatic,SUV,white,1.0,2018-07-16,19,kia,7,2018,2018-08-04,8,4
9,9200,2008,honda pilot,excellent,6.0,gas,147191.0,automatic,SUV,blue,1.0,2019-02-15,17,honda,2,2019,2019-03-04,3,4


Note: I didn't end up using the columns with the year/month/date posted values since it didn't offer any new information and the plot obtained with them weren't very interesting. It seems like in general the amount of cars listed by month and even day, didn't vary much.
I also didn't use the "date of purchase" obtained because I am not certain that it is a correct appreciationg. Plus it probably won't create interesting plots, just as it happened with the date-posted data. However, I'm leaving everything on the notebook for documentation purposes.

I thought it would be interesting to create a classification according to the car's age, to identify how many vintage, classic and antique cars are sold. As well as how the age affects their price. 
This classification was based mostly on the information found in the webpage: https://americancollectors.com/articles/vintage-vs-classic-vs-antique-cars/

In [197]:


def age_clasification(row):
    """
    The function returns a category according to the car's year model, using the following rules:
    —'nearly_new' for model_year >= 2017
    —'late_model' for model_year >= 2009 
    —'modern'  for model_year >= 2000
    —'classic' for model_year >= 1974
    —'antique' for year >= 1931 
    —'vintage' for year < 1931

    """

    year = row['model_year']
    
    if year >= 2017:
        return 'nearly_new'
    else:
        if year >= 2009:
            return 'late_model'
        else:
            if year >= 2000:
                return 'modern'
            else:
                if year >= 1974:
                    return 'classic'
                else:
                    if year >= 1931:
                        return 'antique'
                    else:
                        if year < 1931:
                            return 'vintage'
   


df['classification'] = df.apply(age_clasification, axis=1)
df.sample(10)


,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed,make,month_posted,year_posted,date_purchase,month_purchase,day_purchase,classification
35407,6800,2005,chevrolet silverado,good,8.0,gas,175000.0,automatic,pickup,black,0.0,2018-05-20,31,chevrolet,5,2018,2018-06-20,6,20,modern
6845,6995,2011,chrysler 200,good,4.0,gas,86035.0,automatic,convertible,silver,0.0,2018-07-05,5,chrysler,7,2018,2018-07-10,7,10,late_model
50937,8900,2007,gmc yukon,good,8.0,gas,158454.0,automatic,SUV,blue,1.0,2019-01-28,5,gmc,1,2019,2019-02-02,2,2,modern
12481,4795,2008,chevrolet equinox,excellent,6.0,gas,107600.0,automatic,SUV,black,1.0,2018-07-12,38,chevrolet,7,2018,2018-08-19,8,19,modern
17142,6995,2007,toyota camry,like new,6.0,gas,114000.0,automatic,sedan,red,0.0,2019-03-13,35,toyota,3,2019,2019-04-17,4,17,modern
5899,18000,2017,dodge grand caravan,excellent,6.0,gas,15538.0,automatic,mini-van,red,0.0,2018-05-03,24,dodge,5,2018,2018-05-27,5,27,nearly_new
27574,4995,2000,ford f-150,good,8.0,gas,241915.0,automatic,truck,white,0.0,2019-01-06,47,ford,1,2019,2019-02-22,2,22,modern
27028,8499,2011,toyota camry,good,4.0,gas,85000.0,automatic,sedan,silver,0.0,2018-05-08,59,toyota,5,2018,2018-07-06,7,6,late_model
7478,6995,2005,ram 1500,excellent,8.0,gas,171713.0,automatic,pickup,red,1.0,2018-05-15,37,ram,5,2018,2018-06-21,6,21,modern
37866,11900,2014,toyota prius,excellent,4.0,hybrid,108000.0,automatic,hatchback,red,0.0,2019-01-02,39,toyota,1,2019,2019-02-10,2,10,late_model


1.5 Designing the web app
In this section we will create the information to be displayed in the app. 

In [198]:
st.header('Used cars advertisements')
st.dataframe(df)

DeltaGenerator()

In [199]:
#Creating a bar chart comparing the amount of cars advertised per year/month and make
adv_year_month=df.groupby(['year_posted', 'month_posted'])['make'].value_counts().reset_index(name='count')
print(adv_year_month)



     year_posted  month_posted           make  count
0           2018             5           ford   1112
1           2018             5      chevrolet    900
2           2018             5         toyota    475
3           2018             5          honda    335
4           2018             5            ram    303
..           ...           ...            ...    ...
222         2019             4       cadillac     18
223         2019             4          acura     14
224         2019             4          buick     12
225         2019             4            bmw     11
226         2019             4  mercedes-benz      5

[227 rows x 4 columns]


In [200]:
chart = alt.Chart(adv_year_month).mark_bar().encode(x='month_posted', y='count', color='make', column='year_posted')
chart

c:\Users\Owner\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning:

the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.



alt.Chart(...)

I thought it would be interesting to see a comparison of the amount of cars sold per month/year and make. However, I don't think it provides very useful information except for cleary stating the range of the data we have, and making visual the amount of advitised cars per make. This plot won't be used on the final web service.

In [204]:
#I'll create a bar chart showing the amount of cars per type, according to their classification (age) and make.

type_options = df['type'].unique().tolist()
make = st.selectbox('Which type would you like to see?', type_options, 0)
#df = df[df['type']==type]

fig = px.bar(df, x='classification', color='make')
fig.update_xaxes(type='category')
fig.update_layout()
st.write(fig)

In [205]:

#Histogram of condition vs days_listed
st.header("Histogram of 'condition' vs 'days_listed'" )
make_options = df['make'].unique().tolist()
make = st.selectbox('Which make would you like to see?', make_options, 0)
fig = px.histogram(df, x='days_listed', color='condition')
st.write(fig)


In [ ]:
#Distribution of price according to the vehicle type
st.header('Distribution of price according to the vehicle type')
type_vehicles = sorted(df['type'].unique())
#First dropdown menu
type_1 = st.selectbox(label='Select vehicle type 1', options=type_vehicles, index=type_vehicles.index('sedan'))
#Second dropdown menu
type_2 = st.selectbox(label='Select vehicle type 2', options=type_vehicles, index=type_vehicles.index('SUV'))
#Filtering the dataframe
mask_type = (df['type'] == type_1) | (df['type'] == type_2)
df_masked = df[mask_type]

#Adding a checkbox to normalize histogram
normalize = st.checkbox('Normalize histogram', value=True)
if normalize:
    histnorm = 'percent'
else:
    histnorm = None

#Creating a plotly histogram figure
fig = px.histogram(df_masked, x='price', nbins=25, color='type', histnorm=histnorm, barmode='overlay', color_discrete_sequence=["green", "blue"], opacity=0.5)
st.write(fig)
st.write('It is not a functional application yet. Under construction.')

This way I finished creating all of the plots that I will be using on the web service. Some extra elements will de needed yet I will be creating them directly on the app.py file, since by doing them there I can test them directly via the localhost. They will mainly be text elements to clarify what is shown there.

1.6 Conclussions

Some relevant conclussions drawn by the analysis are:
-It wasn't possible to compare the information of advertisements posted on each different year (2018 and 2019) because the data provided spans one year only (across the two years mentioned).
-Some interesting information was obtained by classifying the vehicles by their age. Most of the cars posted correspond to the 'late_model' classification, followed by the 'modern' and on a third place are the 'nearly_new".
-It is expected that the least amount of cars would correspond to the older categories. However, I was surprised to see that there are some interesting vintage cars sold. 
-It is also apparent that the make with the most presence on the advertisements is Ford, followed by Chevrolet and Toyota.
-In regards to the car condition, as stated on the advertisements, the majority of them claim to be in excelent conditions and on the opposite side, only a few claim to be either salvage or new.